In [1]:
# from __future__ import print_function

import sys

import numpy as np
from pyspark.sql import SparkSession
from datetime import datetime, timedelta
import pyspark.sql.functions as F

import pytz
# print(__name__)builtins
if __name__ == "builtins":
    spark = SparkSession \
        .builder \
        .appName("Python Demo") \
        .config("hive.metastore.client.factory.class",
                "com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory") \
        .config("spark.driver.maxResultSize", "4g") \
        .enableHiveSupport() \
        .getOrCreate()
    spark.conf.set("hive.exec.dynamic.partition.mode", "nonstrict");
    spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
    spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "20000000")
    databases = spark.sql("show databases")
    databases = databases.collect()
    tablelist=["`banda-etl-s3`.`t_channel_details`","`banda-etl-s3`.`t_customer_install_info`","`banda-etl-s3`.`t_customer`"];
    df1=spark.sql("""
                     with assign_detail as 
            (
            SELECT  audit.admin_id
                   ,audit.collector_id
                   ,audit.trigger
                   ,lpay.id
                   ,lpay.erase_amount
                   ,audit.full_name_x
                   ,audit.loan_app_id
                   ,audit.create_time
                   ,audit.assign_stage
                   ,app.amount
                   ,lpay.due_date
                   ,datediff(DATE(audit.create_time+interval'7'hour),DATE(lpay.due_date+interval'7'hour)) AS overduedays_assign --分案时逾期天数
            FROM 
            (
                SELECT  *
                FROM banda_rpt_mid.t_assign_detail_stage_daily 
                WHERE rn_total_desc=1 --一个案件多次分案，仅计算最后一次分案记录作为在库案件记录 audit
                AND datediff(DATE(now()+interval'7'hour),DATE(create_time+interval'7'hour))>=0 
                AND datediff(DATE(now()+interval'7'hour), DATE(create_time + interval'2'hour+interval'7'hour))<DAY(now()+interval'7'hour) 
            ) audit
            LEFT JOIN `banda-etl-s3`.t_loan_app app
            ON app.id=audit.loan_app_id
            LEFT JOIN `banda-etl-s3`.t_lpay lpay
            ON lpay.loan_app_id=audit.loan_app_id
            where datediff( DATE(audit.create_time+interval'7' hour), DATE(lpay.due_date+interval'7' hour))>-4 -- 分案日>到期日-4天
            ),
            paid_off_id as 
            (
            SELECT  app2.id
            FROM `banda-etl-s3`.t_loan_app app2
            LEFT JOIN `banda-etl-s3`.t_loan_app_status_log lg
            ON lg.loan_app_id=app2.id
            WHERE app2.product_name='RUPIAHONE' 
            AND lg.create_time+interval'7'hour >now()+INTERVAL '-24' hour +INTERVAL'-2' month +interval'7'hour
            AND lg.new_status='PAID_OFF' 
            AND app2.status IN ('CURRENT', 'GRACE_PERIOD', 'OVERDUE', 'PAID_OFF') 
            AND datediff( DATE(now()+interval'7'hour ), DATE(lg.create_time+interval'7'hour ))>0  
            )


            -- 创建当下在库案件量增量表，每日插入当日数据，保留之前数据
            -- banda_rpt_mid.t_case_in_collection_increment 
            SELECT  date_format(now()+INTERVAL'7' hour,'yyyy-MM-dd') AS DATE 
                   ,t3.collector_id 
                   ,t3.full_name_x 
                   ,assign_stage
                   ,COUNT(DISTINCT t3.loan_app_id)                                                                                                         AS count 
                   ,SUM(CASE WHEN t3.principal_paid IS NULL THEN t3.amount ELSE t3.amount-t3.principal_paid END )                                          AS unpaid_amoun
                   ,now() as insert_time
            FROM 
            (
                SELECT  t1.loan_app_id 
                       ,t1.collector_id 
                       ,t1.full_name_x 
                       ,t1.due_date 
                       ,t1.amount 
                       ,t1.assign_stage
                       ,SUM(clear.principal) AS principal_paid
                FROM assign_detail t1
                LEFT JOIN paid_off_id t2
                ON t2.id=t1.loan_app_id
                LEFT JOIN `banda-etl-s3`.t_lpay_deposit deposit
                ON deposit.lpay_id=t1.id AND deposit.status='CLEARED' AND deposit.create_time<t1.create_time AND 
                ( deposit.deposit_method !='DIRECT_TRANSFER' OR (deposit.deposit_method ='DIRECT_TRANSFER' AND deposit.cleared_amount !=ceil(t1.erase_amount)))
                LEFT JOIN `banda-etl-s3`.t_reduce reduce
                ON reduce.deposit_id =deposit.id AND reduce.reduce_status='SUCCEED'
                LEFT JOIN `banda-etl-s3`.t_clear_detail_log clear
                ON clear.deposit_id=deposit.id
                WHERE  t2.id IS NULL 
                AND (reduce.reduce_type IS NULL OR reduce.reduce_type NOT IN ('DELAY_CALLBACK', 'NONE_CALLBACK_REDUCE', 'COLLECTION_REDUCE')) 
                GROUP BY  t1.loan_app_id 
                         ,t1.collector_id 
                         ,t1.full_name_x 
                         ,t1.due_date 
                         ,t1.amount 
                         ,t1.assign_stage
            )t3
            GROUP BY  date_format(now()+INTERVAL'7' hour,'yyyy-MM-dd') 
                     ,t3.collector_id 
                     ,t3.full_name_x 
                     ,assign_stage""")
                  
    df1.repartition(1).write.mode("append").orc("s3://rupiahplus-data-warehouse/etl/banda/report_temp_table/t_case_in_collection_increment/") 
    df2=spark.sql("""-- 新建表：banda_rpt_mid.coll_outsouring_detail_daily
                    -- 增量更新，每日新增数据进去，之前数据保留
                    SELECT  date(now()+interval '-1'day+ INTERVAL '7' hour) AS DATE
                           ,t4.collector_id
                           ,t4.full_name
                           ,t4.overdue
                           ,t4.gp_name
                           ,COUNT(DISTINCT t4.loan_app_id) AS count
                           ,SUM(t4.unpaid_amount)          AS unpaid_amount
                           ,SUM(t4.unpaid_total)           AS unpaid_total
                           ,now() as insert_time
                    FROM
                    (
                        SELECT  t3.collector_id
                               ,t3.full_name
                               ,t3.overdue
                               ,t3.gp_name
                               ,t3.loan_app_id
                               ,t3.create_time
                               ,t3.amount
                               ,t3.clear_amount
                               ,t3.clear_principal
                               ,CASE WHEN t3.clear_principal IS NULL THEN t3.amount  ELSE t3.amount-t3.clear_principal END AS unpaid_amount
                               ,CASE WHEN t3.clear_total IS NULL THEN t3.total_accr  ELSE t3.total_accr-t3.clear_total END AS unpaid_total
                               ,row_number()OVER (PARTITION BY t3.loan_app_id,t3.gp_name ORDER BY t3.create_time ASC )as num2--一个组第一次分案记录
                        FROM
                        (
                            SELECT  t2.collector_id
                                   ,t2.full_name
                                   ,t2.overdue
                                   ,t2.gp_name
                                   ,t2.loan_app_id
                                   ,t2.amount
                                   ,t2.total_accr
                                   ,t2.create_time
                                   ,SUM(deposit.cleared_amount)                                         AS clear_amount
                                   ,SUM(clear.principal)                                                AS clear_principal
                                   ,SUM(clear.principal+clear.interest+clear.penalty+clear.default_fee) AS clear_total
                            FROM
                            (
                                SELECT  t1.loan_app_id
                                       ,t1.collector_id
                                       ,t1.full_name
                                       ,t1.gp_name
                                       ,t1.overdue
                                       ,t1.amount
                                       ,t1.total_accr
                                       ,t1.create_time
                                       ,t1.id
                                       ,t1.erase_amount
                                       ,row_number()OVER (PARTITION BY t1.loan_app_id,t1.overdue ORDER BY t1.create_time DESC )AS num--一个逾期阶段最后一次分案记录
                                FROM
                                (
                                    SELECT  audit.admin_id
                                           ,audit.collector_id
                                           ,ad.full_name_x                                                                                                                 AS full_name
                                           ,audit.loan_app_id
                                           ,audit.create_time
                                           ,app.amount
                                           ,lpay.principal_accr+lpay.interest_accr+lpay.default_accr+lpay.default_fee_accr                                                 AS total_accr
                                           ,lpay.due_date
                                           ,lpay.id
                                           ,lpay.erase_amount
                                           ,substr(depart.name,-2)                                                                                                         AS gp_name
                                           ,CASE WHEN datediff(date(audit.create_time+interval'7'hour),date(lpay.due_date+interval'7'hour))>179 THEN 180
                                                 WHEN datediff(date(audit.create_time+interval'7'hour),date(lpay.due_date+interval'7'hour))>89 THEN 90
                                                 WHEN datediff(date(audit.create_time+interval'7'hour),date(lpay.due_date+interval'7'hour))>60 THEN 61
                                                 WHEN datediff(date(audit.create_time+interval'7'hour),date(lpay.due_date+interval'7'hour))>31 THEN 32  ELSE 31 END AS overdue
                                    FROM banda_rpt_mid.t_assign_detail_stage_daily audit
                                    LEFT JOIN `banda-etl-s3`.t_admin ad
                                    ON ad.id=audit.collector_id
                                    LEFT JOIN `banda-etl-s3`.t_admin_group ad_group
                                    ON ad_group.admin_id=audit.collector_id AND ad_group.status='ACTIVE'
                                    LEFT JOIN `banda-etl-s3`.t_department_group depart
                                    ON depart.id=ad_group.group_id AND depart.status='ACTIVE'
                                    LEFT JOIN `banda-etl-s3`.t_loan_app app
                                    ON app.id=audit.loan_app_id
                                    LEFT JOIN `banda-etl-s3`.t_lpay lpay
                                    ON lpay.loan_app_id=audit.loan_app_id
                                    WHERE datediff(date(now()+ INTERVAL '7' hour), date(audit.create_time+ INTERVAL '7' hour) )>0
                                    AND datediff(date(now()+INTERVAL'-24' hour+ INTERVAL '7' hour), date(audit.create_time + INTERVAL '2' hour+ INTERVAL '7' hour))<day(now()+INTERVAL'-24'hour+ INTERVAL '7' hour)
                                    AND datediff( date(audit.create_time + INTERVAL '2' hour+ INTERVAL '7' hour), date(lpay.due_date+ INTERVAL '7' hour))>30
                                    AND depart.name is NOT NULL
                                )t1
                            )t2
                            LEFT JOIN `banda-etl-s3`.t_lpay_deposit deposit
                            ON deposit.lpay_id=t2.id AND deposit.status='CLEARED' AND deposit.create_time<t2.create_time 
                            AND ( deposit.deposit_method !='DIRECT_TRANSFER' OR (deposit.deposit_method ='DIRECT_TRANSFER' AND deposit.cleared_amount !=ceil(t2.erase_amount)))
                            LEFT JOIN `banda-etl-s3`.t_reduce reduce
                            ON reduce.deposit_id =deposit.id AND reduce.reduce_status='SUCCEED'
                            LEFT JOIN `banda-etl-s3`.t_clear_detail_log clear
                            ON clear.deposit_id=deposit.id
                            WHERE t2.num=1
                            AND (reduce.reduce_type IS NULL OR reduce.reduce_type NOT IN ('DELAY_CALLBACK', 'NONE_CALLBACK_REDUCE', 'COLLECTION_REDUCE'))
                            GROUP BY  t2.collector_id
                                     ,t2.full_name
                                     ,t2.overdue
                                     ,t2.gp_name
                                     ,t2.loan_app_id
                                     ,t2.amount
                                     ,t2.total_accr
                                     ,t2.create_time
                        )t3
                    )t4
                    WHERE t4.num2=1
                    GROUP BY  date(now()+interval '-1'day+ INTERVAL '7' hour)
                             ,t4.collector_id
                             ,t4.full_name
                             ,t4.overdue
                             ,t4.gp_name """)
    df2.repartition(1).write.mode("append").orc("s3://rupiahplus-data-warehouse/etl/banda/report_temp_table/coll_outsouring_detail_daily/")
    df3=spark.sql("""
            -- 创建日分区表 banda_rpt_mid.coll_collector_info_detail_dt
            -- 每日数据存为一个分区保留记录
            -- partition by dt

            SELECT  t2.collector_id
                   ,t2.full_name_x
                   ,substring_index(full_name_x,"-",1) AS collector_name
                   ,assign_stage
                   ,t2.team_leader_name
                   ,group_name
                   ,t2.team_name
                   ,NOW() as insert_time,
                   date(now()) as date_time
            FROM
            (
                SELECT  audit.*
                       ,ad.team_leader_id
                       ,depart.description AS team_name
                       ,depart.name        AS group_name
                       ,depart2.description
                       ,ad2.full_name_x    AS team_leader_name
                FROM
                ( select*from banda_rpt_mid.t_assign_detail_stage_daily
                    WHERE rn_ingroup_desc=1 
                ) audit
                LEFT JOIN `banda-etl-s3`.t_admin ad
                ON ad.id=audit.collector_id
                LEFT JOIN `banda-etl-s3`.t_admin_group ad_group
                ON ad_group.admin_id=audit.collector_id AND ad_group.status= 'ACTIVE'
                LEFT JOIN `banda-etl-s3`.t_department_group depart
                ON depart.id=ad_group.group_id AND depart.status='ACTIVE'
                LEFT JOIN `banda-etl-s3`.t_department_group depart2
                ON depart2.id=depart.parent_id AND depart2.status='ACTIVE'
                LEFT JOIN `banda-etl-s3`.t_admin_group ad_group2
                ON ad_group2.group_id=depart2.id AND ad_group2.status='ACTIVE'
                LEFT JOIN `banda-etl-s3`.t_admin ad2
                ON ad2.id=ad_group2.admin_id
                WHERE depart.name IS NOT NULL
                AND DATEDIFF( DATE(now()+INTERVAL'7'hour), DATE(audit.create_time+INTERVAL'7'hour))>0
                AND DATEDIFF(DATE(now()+INTERVAL'-24' hour+INTERVAL'7'hour), DATE(audit.create_time + interval'2' hour+INTERVAL'7'hour))<DAY(now()+INTERVAL'-24' hour+INTERVAL'7'hour) 
            )t2
            GROUP BY  t2.collector_id
                     ,t2.full_name_x
                     ,substring_index(full_name_x,"-",1)
                     ,assign_stage
                     ,t2.team_leader_name
                     ,group_name
                     ,t2.team_name
                     ,NOW() """)
    df3.repartition(1).write.mode("append").orc("s3://rupiahplus-data-warehouse/etl/banda/report_temp_table/coll_collector_info_detail_dt/")
    df4=spark.sql("""
            -- 创建增量表，催收人员每日工作情况明细每日更新前一日数据
            -- banda_rpt_mid.coll_work_detail_daily
            -- `banda-etl-s3`.t_call_center_info数据不全，采用增量表的方式
            SELECT  date(call_lg.create_time+interval'7'hour) AS DATE
                   ,call_lg.caller_id
                   ,assign_stage
                   ,admin.full_name_x      AS full_name
                   ,COUNT(call_lg.call_id) AS count
                   ,'call_count'AS type

                   ,now()                  AS insert_time
            FROM `banda-etl-s3`.t_call_center_log call_lg
            LEFT JOIN
            (
                SELECT  collector_id
                       ,assign_stage
                       ,full_name_x
                FROM banda_rpt_mid.t_assign_detail_stage_daily
                WHERE rn_total_desc=1--取最后一次分案
                AND date_format(create_time+interval'7'hour,'yy-MM')=date_format(now()+INTERVAL '-1'day,'yy-MM') --取观测日前一天的当月全量分案
                GROUP BY  collector_id
                         ,assign_stage
                         ,full_name_x
            )admin
            ON admin.collector_id=call_lg.caller_id
            WHERE date(call_lg.create_time+interval'7'hour)=DATE(now()+INTERVAL '-1'day+interval'7'hour)
            AND call_lg.caller_id is NOT NULL
            AND admin.full_name_x is NOT NULL
            GROUP BY  date(call_lg.create_time+interval'7'hour)
                     ,call_lg.caller_id
                     ,assign_stage
                     ,admin.full_name_x

            -- 当日log去重量：当日在库案件中标记过log的案件数（每日每个案子仅计算一次）
            UNION ALL

            SELECT  date(rel.create_time+interval'7'hour) AS DATE
                   ,rel.update_id                         AS collector_id
                   ,dd.assign_stage
                   ,dd.full_name_x
                   ,COUNT(distinct rel.loan_id)           AS count
                   ,'log_count'AS type

                   ,now()                                 AS insert_time
            FROM
            (
                SELECT  rel.loan_id
                       ,rel.create_time
                       ,update_id
                FROM `banda-etl-s3`.t_loan_tag_rel rel
                JOIN
                (
                    SELECT  id
                    FROM `banda-etl-s3`.t_tag
                    WHERE type='MANUAL_COLLECTION_LOG'
                    AND status='ACTIVE' 
                )tag
                ON tag.id=rel.tag_id
            ) rel
            LEFT JOIN banda_rpt_mid.t_assign_detail_stage_daily dd
            ON rel.loan_id=dd.loan_app_id AND rel.update_id=dd.collector_id
            WHERE date(rel.create_time+interval'7'hour)=DATE(now()+INTERVAL '-1'day+interval'7'hour)
            AND dd.collector_id is not null
            GROUP BY  date(rel.create_time+interval'7'hour)
                     ,rel.update_id
                     ,dd.assign_stage
                     ,dd.full_name_x

            UNION ALL

            -- 当天主动还款案件量：当天未下log的paid off案件数
            SELECT  date(lg.create_time+interval'7'hour)     AS DATE
                   ,assign.collector_id 
                   ,assign.assign_stage
                   ,assign.full_name_x
                   ,COUNT(DISTINCT assign.loan_app_id)       AS count
                   ,'Initiatively_count'AS type
                   ,now() as insert_time
            FROM
            ( SELECT*
                FROM  banda_rpt_mid.t_assign_detail_stage_daily
                WHERE rn_total_desc=1 
            ) assign
            JOIN
            ( select*from `banda-etl-s3`.t_loan_app
                WHERE product_name='RUPIAHONE'
                AND status='PAID_OFF' 
            ) app
            ON assign.loan_app_id=app.id
            LEFT JOIN
            ( select*from `banda-etl-s3`.t_loan_app_status_log
                WHERE new_status='PAID_OFF' 
            )lg
            ON lg.loan_app_id=app.id
            LEFT JOIN
            (
                SELECT  rel2.loan_id
                       ,update_id
                       ,rel2.create_time AS log_time
                FROM `banda-etl-s3`.t_loan_tag_rel rel2
                JOIN
                (
                    SELECT  id
                    FROM `banda-etl-s3`.t_tag
                    WHERE type='MANUAL_COLLECTION_LOG'
                    AND status='ACTIVE' 
                )tag
                ON tag.id=rel2.tag_id
            )t1
            ON t1.loan_id=assign.loan_app_id AND t1.update_id=assign.collector_id
            LEFT JOIN
            (
                SELECT  rel2.loan_id
                       ,update_id
                       ,rel2.create_time AS log_time
                FROM `banda-etl-s3`.t_loan_tag_rel rel2
                JOIN
                (
                    SELECT  id
                    FROM `banda-etl-s3`.t_tag
                    WHERE type='MANUAL_COLLECTION_LOG'
                    AND status='ACTIVE' 
                )tag
                ON tag.id=rel2.tag_id
            )t2
            ON t2.loan_id=assign.loan_app_id AND date(t2.log_time+interval'7'hour)=date(lg.create_time+interval'7'hour)
            WHERE date(lg.create_time+interval'7'hour)=DATE(now()+INTERVAL '-1'day+interval'7'hour) 
            AND (t1.loan_id IS NULL OR date(t1.log_time+interval'7'hour)<date(lg.create_time+interval'7'hour)) -- 没有催记标记就结清或者LOG当天之后结清
            AND t2.loan_id is null -- 结清当日有催记的剔除（有催记的案件已被计算到当日LOG去重数里面，避免重复计算）
            GROUP BY  date(lg.create_time+interval'7'hour)
                     ,assign.collector_id
                     ,assign.assign_stage
                     ,assign.full_name_x
    """)
    df4.repartition(1).write.mode("append").orc("s3://rupiahplus-data-warehouse/etl/banda/report_temp_table/coll_work_detail_daily/")

VBox()

Starting Spark application


The code failed because of a fatal error:
	Session 6 unexpectedly reached final status 'dead'. See logs:
stdout: 
#
# There is insufficient memory for the Java Runtime Environment to continue.
# Native memory allocation (mmap) failed to map 69599232 bytes for committing reserved memory.
# An error report file with more information is saved as:
# /tmp/hs_err_pid18135.log

stderr: 
21/08/25 10:10:20 INFO RSCDriver: Connecting to: ip-10-3-0-76.ap-southeast-1.compute.internal:10000
21/08/25 10:10:20 INFO RSCDriver: Starting RPC server...
21/08/25 10:10:20 INFO RpcServer: Connected to the port 10002
21/08/25 10:10:20 WARN RSCConf: Your hostname, ip-10-3-0-76.ap-southeast-1.compute.internal, resolves to a loopback address, but we couldn't find any external IP address!
21/08/25 10:10:20 WARN RSCConf: Set livy.rsc.rpc.server.address if you need to bind to another address.
21/08/25 10:10:21 INFO RSCDriver: Received job request 23f0b95a-bfd9-4c98-8202-bf958a21d40d
21/08/25 10:10:21 INFO RSCDrive

In [4]:
spark.sql("""
-- 新建表：banda_rpt_mid.coll_outsouring_detail_daily
-- 增量更新，每日新增数据进去，之前数据保留
SELECT  date(now()+interval '-1'day+ INTERVAL '7' hour) AS DATE
       ,t4.collector_id
       ,t4.full_name
       ,t4.overdue
       ,t4.gp_name
       ,COUNT(DISTINCT t4.loan_app_id) AS count
       ,SUM(t4.unpaid_amount)          AS unpaid_amount
       ,SUM(t4.unpaid_total)           AS unpaid_total
       ,now() as insert_time
FROM
(
	SELECT  t3.collector_id
	       ,t3.full_name
	       ,t3.overdue
	       ,t3.gp_name
	       ,t3.loan_app_id
	       ,t3.create_time
	       ,t3.amount
	       ,t3.clear_amount
	       ,t3.clear_principal
	       ,CASE WHEN t3.clear_principal IS NULL THEN t3.amount  ELSE t3.amount-t3.clear_principal END AS unpaid_amount
	       ,CASE WHEN t3.clear_total IS NULL THEN t3.total_accr  ELSE t3.total_accr-t3.clear_total END AS unpaid_total
	       ,row_number()OVER (PARTITION BY t3.loan_app_id,t3.gp_name ORDER BY t3.create_time ASC )as num2--一个组第一次分案记录
	FROM
	(
		SELECT  t2.collector_id
		       ,t2.full_name
		       ,t2.overdue
		       ,t2.gp_name
		       ,t2.loan_app_id
		       ,t2.amount
		       ,t2.total_accr
		       ,t2.create_time
		       ,SUM(deposit.cleared_amount)                                         AS clear_amount
		       ,SUM(clear.principal)                                                AS clear_principal
		       ,SUM(clear.principal+clear.interest+clear.penalty+clear.default_fee) AS clear_total
		FROM
		(
			SELECT  t1.loan_app_id
			       ,t1.collector_id
			       ,t1.full_name
			       ,t1.gp_name
			       ,t1.overdue
			       ,t1.amount
			       ,t1.total_accr
			       ,t1.create_time
			       ,t1.id
			       ,t1.erase_amount
			       ,row_number()OVER (PARTITION BY t1.loan_app_id,t1.overdue ORDER BY t1.create_time DESC )AS num--一个逾期阶段最后一次分案记录
			FROM
			(
				SELECT  audit.admin_id
				       ,audit.collector_id
				       ,ad.full_name_x                                                                                                                 AS full_name
				       ,audit.loan_app_id
				       ,audit.create_time
				       ,app.amount
				       ,lpay.principal_accr+lpay.interest_accr+lpay.default_accr+lpay.default_fee_accr                                                 AS total_accr
				       ,lpay.due_date
				       ,lpay.id
				       ,lpay.erase_amount
				       ,substr(depart.name,-2)                                                                                                         AS gp_name
				       ,CASE WHEN datediff(date(audit.create_time+interval'7'hour),date(lpay.due_date+interval'7'hour))>179 THEN 180
				             WHEN datediff(date(audit.create_time+interval'7'hour),date(lpay.due_date+interval'7'hour))>89 THEN 90
				             WHEN datediff(date(audit.create_time+interval'7'hour),date(lpay.due_date+interval'7'hour))>60 THEN 61
				             WHEN datediff(date(audit.create_time+interval'7'hour),date(lpay.due_date+interval'7'hour))>31 THEN 32  ELSE 31 END AS overdue
				FROM banda_rpt_mid.t_assign_detail_stage_daily audit
				LEFT JOIN `banda-etl-s3`.t_admin ad
				ON ad.id=audit.collector_id
				LEFT JOIN `banda-etl-s3`.t_admin_group ad_group
				ON ad_group.admin_id=audit.collector_id AND ad_group.status='ACTIVE'
				LEFT JOIN `banda-etl-s3`.t_department_group depart
				ON depart.id=ad_group.group_id AND depart.status='ACTIVE'
				LEFT JOIN `banda-etl-s3`.t_loan_app app
				ON app.id=audit.loan_app_id
				LEFT JOIN `banda-etl-s3`.t_lpay lpay
				ON lpay.loan_app_id=audit.loan_app_id
				WHERE datediff(date(now()+ INTERVAL '7' hour), date(audit.create_time+ INTERVAL '7' hour) )>0
				AND datediff(date(now()+INTERVAL'-24' hour+ INTERVAL '7' hour), date(audit.create_time + INTERVAL '2' hour+ INTERVAL '7' hour))<day(now()+INTERVAL'-24'hour+ INTERVAL '7' hour)
				AND datediff( date(audit.create_time + INTERVAL '2' hour+ INTERVAL '7' hour), date(lpay.due_date+ INTERVAL '7' hour))>30
				AND depart.name is NOT NULL
			)t1
		)t2
		LEFT JOIN `banda-etl-s3`.t_lpay_deposit deposit
		ON deposit.lpay_id=t2.id AND deposit.status='CLEARED' AND deposit.create_time<t2.create_time 
		AND ( deposit.deposit_method !='DIRECT_TRANSFER' OR (deposit.deposit_method ='DIRECT_TRANSFER' AND deposit.cleared_amount !=ceil(t2.erase_amount)))
		LEFT JOIN `banda-etl-s3`.t_reduce reduce
		ON reduce.deposit_id =deposit.id AND reduce.reduce_status='SUCCEED'
		LEFT JOIN `banda-etl-s3`.t_clear_detail_log clear
		ON clear.deposit_id=deposit.id
		WHERE t2.num=1
		AND (reduce.reduce_type IS NULL OR reduce.reduce_type NOT IN ('DELAY_CALLBACK', 'NONE_CALLBACK_REDUCE', 'COLLECTION_REDUCE'))
		GROUP BY  t2.collector_id
		         ,t2.full_name
		         ,t2.overdue
		         ,t2.gp_name
		         ,t2.loan_app_id
		         ,t2.amount
		         ,t2.total_accr
		         ,t2.create_time
	)t3
)t4
WHERE t4.num2=1
GROUP BY  date(now()+interval '-1'day+ INTERVAL '7' hour)
         ,t4.collector_id
         ,t4.full_name
         ,t4.overdue
         ,t4.gp_name
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------+--------------------+------------+-----+-------------------+--------------------+
|      DATE|collector_id|         full_name_x|assign_stage|count|       unpaid_amoun|         insert_time|
+----------+------------+--------------------+------------+-----+-------------------+--------------------+
|2021-08-19|      102213|Octavianus Sukman...|          Q0|  153| 288709999.00000000|2021-08-19 10:12:...|
|2021-08-19|      102428|     Evi Lestasri-Q3|          Q3|  380| 573164000.00000000|2021-08-19 10:12:...|
|2021-08-19|      102375|Christopher Louis...|          Q2|  172| 271515000.00000000|2021-08-19 10:12:...|
|2021-08-19|      102167|Wiwin Wijayanto - Q2|          Q2|  171| 269565200.00000000|2021-08-19 10:12:...|
|2021-08-19|      102411|Wiwit Aulia Suwar...|          Q0|  153| 291488000.00000000|2021-08-19 10:12:...|
|2021-08-19|      102042|Adhitya Syahputra-Q3|          Q3|  380| 565870600.00000000|2021-08-19 10:12:...|
|2021-08-19|      102435|    Ayu Pebr

########补充历史

In [3]:
df=spark.sql("""

    select * from zhusu.temp_current_case_0801  union 
    select * from zhusu.temp_current_case_0802 union
    select * from zhusu.temp_current_case_0803 union
    select * from zhusu.temp_current_case_0804  union
    select * from zhusu.temp_current_case_0805 union
    select * from zhusu.temp_current_case_0806 union
    select * from zhusu.temp_current_case_0807 union
    select * from zhusu.temp_current_case_0808 union 
    select * from zhusu.temp_current_case_0809  union
    select * from  zhusu.temp_current_case_0810 union
    select * from zhusu.temp_current_case_0811 union
    select * from zhusu.temp_current_case_0812 union
    select * from zhusu.temp_current_case_0813 union
    select * from zhusu.temp_current_case_0814 union
    select * from zhusu.temp_current_case_0815 

""")
df.repartition(1).write.mode("append").orc("s3://rupiahplus-data-warehouse/etl/banda/report_temp_table/t_case_in_collection_increment/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
    df1=spark.sql("""
            -- 创建日分区表 banda_rpt_mid.coll_collector_info_detail_dt
            -- 每日数据存为一个分区保留记录
            -- partition by dt

            SELECT  t2.collector_id
                   ,t2.full_name_x
                   ,substring_index(full_name_x,"-",1) AS collector_name
                   ,assign_stage
                   ,t2.team_leader_name
                   ,group_name
                   ,t2.team_name
                   ,NOW() as insert_time,
                   date(now()) as date_time
            FROM
            (
                SELECT  audit.*
                       ,ad.team_leader_id
                       ,depart.description AS team_name
                       ,depart.name        AS group_name
                       ,depart2.description
                       ,ad2.full_name_x    AS team_leader_name
                FROM
                ( select*from banda_rpt_mid.t_assign_detail_stage_daily
                    WHERE rn_ingroup_desc=1 
                ) audit
                LEFT JOIN `banda-etl-s3`.t_admin ad
                ON ad.id=audit.collector_id
                LEFT JOIN `banda-etl-s3`.t_admin_group ad_group
                ON ad_group.admin_id=audit.collector_id AND ad_group.status= 'ACTIVE'
                LEFT JOIN `banda-etl-s3`.t_department_group depart
                ON depart.id=ad_group.group_id AND depart.status='ACTIVE'
                LEFT JOIN `banda-etl-s3`.t_department_group depart2
                ON depart2.id=depart.parent_id AND depart2.status='ACTIVE'
                LEFT JOIN `banda-etl-s3`.t_admin_group ad_group2
                ON ad_group2.group_id=depart2.id AND ad_group2.status='ACTIVE'
                LEFT JOIN `banda-etl-s3`.t_admin ad2
                ON ad2.id=ad_group2.admin_id
                WHERE depart.name IS NOT NULL
                AND DATEDIFF( DATE(now()+INTERVAL'7'hour), DATE(audit.create_time+INTERVAL'7'hour))>0
                AND DATEDIFF(DATE(now()+INTERVAL'-24' hour+INTERVAL'7'hour), DATE(audit.create_time + interval'2' hour+INTERVAL'7'hour))<DAY(now()+INTERVAL'-24' hour+INTERVAL'7'hour) 
            )t2
            GROUP BY  t2.collector_id
                     ,t2.full_name_x
                     ,substring_index(full_name_x,"-",1)
                     ,assign_stage
                     ,t2.team_leader_name
                     ,group_name
                     ,t2.team_name
                     ,NOW() """)
    df1.repartition(1).write.mode("append").orc("s3://rupiahplus-data-warehouse/etl/banda/report_temp_table/coll_collector_info_detail_dt11/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…